This is a fork of [best-public-score](https://www.kaggle.com/code/peizhengwang/best-public-score) notebook with some tuned hyperparameters for LightGBM model, that result in a slight improvement of LB score: 5.3365 -> 5.3359. All the credits go to the original author @peizhengwang.

In [1]:
import gc  
import os  
import time  
import warnings 
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
import polars as pl
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_offline = False 
LGB = False
CAT_train=False
CAT=True
NN = False
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# **Utilities**

In [2]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

In [3]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
    return df


# Data Loading






In [4]:
df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = df.dropna(subset=["target"])
df.reset_index(drop=True, inplace=True)
df_shape = df.shape

**Outlier removal**

 # Parallel Triplet Imbalance Calculation

In [5]:
from numba import njit, prange

@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features


# Feature Generation Functions 

In [6]:
def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
   
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    df['spread_depth_ratio'] = (df['ask_price'] - df['bid_price']) / (df['bid_size'] + df['ask_size'])
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    
    df['micro_price'] = ((df['bid_price'] * df['ask_size']) + (df['ask_price'] * df['bid_size'])) / (df['bid_size'] + df['ask_size'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
    
    # Calculate various statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        

    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1,2,3,4,5,6,7,8,9,10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'weighted_wap','price_spread']:
        for window in [1,2,3,4,5,6,7,8,9,10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    
    #V4 feature
    for window in [2,3,4,5,6,7,8,9,10]:
        df[f'price_change_diff_{window}'] = df[f'bid_price_diff_{window}'] - df[f'ask_price_diff_{window}']
        df[f'size_change_diff_{window}'] = df[f'bid_size_diff_{window}'] - df[f'ask_size_diff_{window}']

    #V5 - rolling diff
    # Convert from pandas to Polars
    pl_df = pl.from_pandas(df)

    #Define the windows and columns for which you want to calculate the rolling statistics
    windows = [2,3,4,5,6,7,8,9,10]
    columns = ['ask_price', 'bid_price', 'ask_size', 'bid_size']

    # prepare the operations for each column and window
    group = ["stock_id"]
    expressions = []

    # Loop over each window and column to create the rolling mean and std expressions
    for window in windows:
        for col in columns:
            rolling_mean_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_mean(window)
                .over(group)
                .alias(f'rolling_diff_{col}_{window}')
            )

            rolling_std_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_std(window)
                .over(group)
                .alias(f'rolling_std_diff_{col}_{window}')
            )

            expressions.append(rolling_mean_expr)
            expressions.append(rolling_std_expr)

    # Run the operations using Polars' lazy API
    lazy_df = pl_df.lazy().with_columns(expressions)

    # Execute the lazy expressions and overwrite the pl_df variable
    pl_df = lazy_df.collect()

    # Convert back to pandas if necessary
    df = pl_df.to_pandas()
    gc.collect()
    
    df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    
    for col in df.columns:
        df[col] = df[col].replace([np.inf, -np.inf], 0)

    return df

def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  
    df["minute"] = df["seconds_in_bucket"] // 60  
    df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

def generate_all_features(df):
    # Select relevant columns for feature generation
    cols = [c for c in df.columns if c not in ["row_id", "time_id", "target"]]
    df = df[cols]
    
    # Generate imbalance features
    df = imbalance_features(df)
    gc.collect() 
    df = other_features(df)
    gc.collect()  
    feature_name = [i for i in df.columns if i not in ["row_id", "target", "time_id", "date_id"]]
    
    return df[feature_name]


In [7]:
weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
weights = {int(k):v for k,v in enumerate(weights)}

## Data Splitting

In [8]:
if is_offline:
    
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
    
else:
    df_train = df
    print("Online mode")


Online mode


In [9]:
if is_train:
    feature_columns=['stock_id',
     'seconds_in_bucket',
     'imbalance_size',
     'imbalance_buy_sell_flag',
     'reference_price',
     'matched_size',
     'far_price',
     'near_price',
     'bid_price',
     'bid_size',
     'ask_price',
     'ask_size',
     'wap',
     'volume',
     'mid_price',
     'liquidity_imbalance',
     'matched_imbalance',
     'size_imbalance',
     'reference_price_far_price_imb',
     'reference_price_near_price_imb',
     'reference_price_ask_price_imb',
     'reference_price_bid_price_imb',
     'reference_price_wap_imb',
     'far_price_near_price_imb',
     'far_price_ask_price_imb',
     'far_price_bid_price_imb',
     'far_price_wap_imb',
     'near_price_ask_price_imb',
     'near_price_bid_price_imb',
     'near_price_wap_imb',
     'ask_price_bid_price_imb',
     'ask_price_wap_imb',
     'bid_price_wap_imb',
     'ask_price_bid_price_wap_imb2',
     'ask_price_bid_price_reference_price_imb2',
     'ask_price_wap_reference_price_imb2',
     'bid_price_wap_reference_price_imb2',
     'matched_size_bid_size_ask_size_imb2',
     'matched_size_bid_size_imbalance_size_imb2',
     'matched_size_ask_size_imbalance_size_imb2',
     'bid_size_ask_size_imbalance_size_imb2',
     'stock_weights',
     'weighted_wap',
     'wap_momentum',
     'imbalance_momentum',
     'price_spread',
     'spread_intensity',
     'price_pressure',
     'market_urgency',
     'depth_pressure',
     'spread_depth_ratio',
     'mid_price_movement',
     'micro_price',
     'relative_spread',
     'all_prices_mean',
     'all_sizes_mean',
     'all_prices_std',
     'all_sizes_std',
     'all_prices_skew',
     'all_sizes_skew',
     'all_prices_kurt',
     'all_sizes_kurt',
     'matched_size_shift_1',
     'matched_size_ret_1',
     'matched_size_shift_2',
     'matched_size_ret_2',
     'matched_size_shift_3',
     'matched_size_ret_3',
     'matched_size_shift_4',
     'matched_size_ret_4',
     'matched_size_shift_5',
     'matched_size_ret_5',
     'matched_size_shift_6',
     'matched_size_ret_6',
     'matched_size_shift_7',
     'matched_size_ret_7',
     'matched_size_shift_8',
     'matched_size_ret_8',
     'matched_size_shift_9',
     'matched_size_ret_9',
     'matched_size_shift_10',
     'matched_size_ret_10',
     'imbalance_size_shift_1',
     'imbalance_size_ret_1',
     'imbalance_size_shift_2',
     'imbalance_size_ret_2',
     'imbalance_size_shift_3',
     'imbalance_size_ret_3',
     'imbalance_size_shift_4',
     'imbalance_size_ret_4',
     'imbalance_size_shift_5',
     'imbalance_size_ret_5',
     'imbalance_size_shift_6',
     'imbalance_size_ret_6',
     'imbalance_size_shift_7',
     'imbalance_size_ret_7',
     'imbalance_size_shift_8',
     'imbalance_size_ret_8',
     'imbalance_size_shift_9',
     'imbalance_size_ret_9',
     'imbalance_size_shift_10',
     'imbalance_size_ret_10',
     'reference_price_shift_1',
     'reference_price_ret_1',
     'reference_price_shift_2',
     'reference_price_ret_2',
     'reference_price_shift_3',
     'reference_price_ret_3',
     'reference_price_shift_4',
     'reference_price_ret_4',
     'reference_price_shift_5',
     'reference_price_ret_5',
     'reference_price_shift_6',
     'reference_price_ret_6',
     'reference_price_shift_7',
     'reference_price_ret_7',
     'reference_price_shift_8',
     'reference_price_ret_8',
     'reference_price_shift_9',
     'reference_price_ret_9',
     'reference_price_shift_10',
     'reference_price_ret_10',
     'imbalance_buy_sell_flag_shift_1',
     'imbalance_buy_sell_flag_ret_1',
     'imbalance_buy_sell_flag_shift_2',
     'imbalance_buy_sell_flag_ret_2',
     'imbalance_buy_sell_flag_shift_3',
     'imbalance_buy_sell_flag_ret_3',
     'imbalance_buy_sell_flag_shift_4',
     'imbalance_buy_sell_flag_ret_4',
     'imbalance_buy_sell_flag_shift_5',
     'imbalance_buy_sell_flag_ret_5',
     'imbalance_buy_sell_flag_shift_6',
     'imbalance_buy_sell_flag_ret_6',
     'imbalance_buy_sell_flag_shift_7',
     'imbalance_buy_sell_flag_ret_7',
     'imbalance_buy_sell_flag_shift_8',
     'imbalance_buy_sell_flag_ret_8',
     'imbalance_buy_sell_flag_shift_9',
     'imbalance_buy_sell_flag_ret_9',
     'imbalance_buy_sell_flag_shift_10',
     'imbalance_buy_sell_flag_ret_10',
     'ask_price_diff_1',
     'ask_price_diff_2',
     'ask_price_diff_3',
     'ask_price_diff_4',
     'ask_price_diff_5',
     'ask_price_diff_6',
     'ask_price_diff_7',
     'ask_price_diff_8',
     'ask_price_diff_9',
     'ask_price_diff_10',
     'bid_price_diff_1',
     'bid_price_diff_2',
     'bid_price_diff_3',
     'bid_price_diff_4',
     'bid_price_diff_5',
     'bid_price_diff_6',
     'bid_price_diff_7',
     'bid_price_diff_8',
     'bid_price_diff_9',
     'bid_price_diff_10',
     'ask_size_diff_1',
     'ask_size_diff_2',
     'ask_size_diff_3',
     'ask_size_diff_4',
     'ask_size_diff_5',
     'ask_size_diff_6',
     'ask_size_diff_7',
     'ask_size_diff_8',
     'ask_size_diff_9',
     'ask_size_diff_10',
     'bid_size_diff_1',
     'bid_size_diff_2',
     'bid_size_diff_3',
     'bid_size_diff_4',
     'bid_size_diff_5',
     'bid_size_diff_6',
     'bid_size_diff_7',
     'bid_size_diff_8',
     'bid_size_diff_9',
     'bid_size_diff_10',
     'weighted_wap_diff_1',
     'weighted_wap_diff_2',
     'weighted_wap_diff_3',
     'weighted_wap_diff_4',
     'weighted_wap_diff_5',
     'weighted_wap_diff_6',
     'weighted_wap_diff_7',
     'weighted_wap_diff_8',
     'weighted_wap_diff_9',
     'weighted_wap_diff_10',
     'price_spread_diff_1',
     'price_spread_diff_2',
     'price_spread_diff_3',
     'price_spread_diff_4',
     'price_spread_diff_5',
     'price_spread_diff_6',
     'price_spread_diff_7',
     'price_spread_diff_8',
     'price_spread_diff_9',
     'price_spread_diff_10',
     'price_change_diff_2',
     'size_change_diff_2',
     'price_change_diff_3',
     'size_change_diff_3',
     'price_change_diff_4',
     'size_change_diff_4',
     'price_change_diff_5',
     'size_change_diff_5',
     'price_change_diff_6',
     'size_change_diff_6',
     'price_change_diff_7',
     'size_change_diff_7',
     'price_change_diff_8',
     'size_change_diff_8',
     'price_change_diff_9',
     'size_change_diff_9',
     'price_change_diff_10',
     'size_change_diff_10',
     'rolling_diff_ask_price_3',
     'rolling_std_diff_ask_price_3',
     'rolling_diff_bid_price_3',
     'rolling_std_diff_bid_price_3',
     'rolling_diff_ask_size_3',
     'rolling_std_diff_ask_size_3',
     'rolling_diff_bid_size_3',
     'rolling_std_diff_bid_size_3',
     'rolling_diff_ask_price_5',
     'rolling_std_diff_ask_price_5',
     'rolling_diff_bid_price_5',
     'rolling_std_diff_bid_price_5',
     'rolling_diff_ask_size_5',
     'rolling_std_diff_ask_size_5',
     'rolling_diff_bid_size_5',
     'rolling_std_diff_bid_size_5',
     'rolling_diff_ask_price_10',
     'rolling_std_diff_ask_price_10',
     'rolling_diff_bid_price_10',
     'rolling_std_diff_bid_price_10',
     'rolling_diff_ask_size_10',
     'rolling_std_diff_ask_size_10',
     'rolling_diff_bid_size_10',
     'rolling_std_diff_bid_size_10',
     'mid_price*volume',
     'harmonic_imbalance',
     'dow',
     'seconds',
     'minute',
     'time_to_market_close',
     'global_median_size',
     'global_std_size',
     'global_ptp_size',
     'global_median_price',
     'global_std_price',
     'global_ptp_price']
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }
#     if is_offline:
#         df_train_feats = generate_all_features(df_train)
#         print("Build Train Feats Finished.")
#         df_valid_feats = generate_all_features(df_valid)
#         print("Build Valid Feats Finished.")
#         df_valid_feats = reduce_mem_usage(df_valid_feats)
#     else:
#         df_train_feats = generate_all_features(df_train)
#         print("Build Online Train Feats Finished.")

#     df_train_feats = reduce_mem_usage(df_train_feats)
    del df_train
    gc.collect()

# **Model Training**

# **CAT**

In [10]:
from catboost import CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm
if CAT_train:
    import numpy as np
    import lightgbm as lgb
    
    cat_params = dict(iterations=7000,
                      learning_rate=0.01,
                      depth=11,
                      l2_leaf_reg=30,
                      bootstrap_type='Bernoulli',
                      subsample=0.66,
                      loss_function='MAE',
                      eval_metric = 'MAE',
                      metric_period=100,
                      od_type='Iter',
                      od_wait=30,
                      task_type='GPU',
                      allow_writing_files=False,
                      )

    feature_columns = list(df_train_feats.columns)
    print(f"Features = {len(feature_columns)}")
    #print(f"Feature length = {len(feature_columns)}")

    num_folds = 5
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues_cat' 
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train_feats[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train_feats[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

        print(f"Fold {i+1} Model Training")

        # Train a LightGBM model for the current fold
        cat_model = CatBoostRegressor(**cat_params)
        cat_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            early_stopping_rounds=100
        )
        


        models.append(cat_model)
        
        
        # Save the model to a file
        #model_filename = os.path.join(model_save_path, f'doblez_{i+1}.model')
        #joblib.dump(cat_model, model_filename)
        
        
        
        #model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        #lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        fold_predictions = cat_model.predict(df_fold_valid[feature_columns])
        fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
        scores.append(fold_score)
        print(f":LGB Fold {i+1} MAE: {fold_score}")
        #------------CBT--------------#
#         fold_predictions = cbt_model.predict(df_fold_valid[feature_columns])
#         fold_score_cbt = mean_absolute_error(fold_predictions, df_fold_valid_target)
#         scores.append(fold_score_cbt)
#         print(f"CBT Fold {i+1} MAE: {fold_score_cbt}")

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

    # Update the lgb_params with the average best iteration
    final_model_params = cat_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1

    for i in range(num_model):
        final_model = CatBoostRegressor(**final_model_params)
        final_model.fit(
            df_train_feats[feature_columns],
            df_train['target'],

        )
        #model_filename = os.path.join(model_save_path, f'total_{i+1}.model')
        #joblib.dump(cat_model, model_filename)
        
        # Append the final model to the list of models
        models.append(final_model)


# **LGB**

In [11]:

if LGB:
    import numpy as np
    import lightgbm as lgb
    
    lgb_params = {
        "objective": "mae",
        "n_estimators": 6000,
        "num_leaves": 256,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
#         "learning_rate": 0.00871,
        "learning_rate": 0.01,
        'max_depth': 11,
        "n_jobs": 4,
        "device": "gpu",
        "verbosity": -1,
        "importance_type": "gain",
#         "reg_alpha": 0.1,
        "reg_alpha": 0.2,
        "reg_lambda": 3.25
    }

    feature_columns = list(df_train_feats.columns)
    print(f"Features = {len(feature_columns)}")
    #print(f"Feature length = {len(feature_columns)}")

    num_folds = 5
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues' 
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train_feats[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train_feats[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

        print(f"Fold {i+1} Model Training")

        # Train a LightGBM model for the current fold
        lgb_model = lgb.LGBMRegressor(**lgb_params)
        lgb_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )
        
#         cbt_model = cbt.CatBoostRegressor(objective='MAE', iterations=5000,bagging_temperature=0.5,
#                                 colsample_bylevel = 0.7,learning_rate = 0.065,
#                                 od_wait = 25,max_depth = 7,l2_leaf_reg = 1.5,
#                                 min_data_in_leaf = 1000,random_strength=0.65,
#                                 verbose=0,use_best_model=True,task_type='CPU')
#         cbt_model.fit(
#             df_fold_train[feature_columns],
#             df_fold_train_target,
#             eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)]
#         )
        
#         models_cbt.append(cbt_model)

        models.append(lgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        fold_predictions = lgb_model.predict(df_fold_valid[feature_columns])
        fold_score = mean_absolute_error(fold_predictions, df_fold_valid_target)
        scores.append(fold_score)
        print(f":LGB Fold {i+1} MAE: {fold_score}")
        #------------CBT--------------#
#         fold_predictions = cbt_model.predict(df_fold_valid[feature_columns])
#         fold_score_cbt = mean_absolute_error(fold_predictions, df_fold_valid_target)
#         scores.append(fold_score_cbt)
#         print(f"CBT Fold {i+1} MAE: {fold_score_cbt}")

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

    # Update the lgb_params with the average best iteration
    final_model_params = lgb_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1

    for i in range(num_model):
        final_model = lgb.LGBMRegressor(**final_model_params)
        final_model.fit(
            df_train_feats[feature_columns],
            df_train['target'],
            callbacks=[
                lgb.callback.log_evaluation(period=100),
            ],
        )
        # Append the final model to the list of models
        models.append(final_model)


In [12]:
select_feature=['seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'volume',
 'mid_price',
 'liquidity_imbalance',
 'matched_imbalance',
 'size_imbalance',
 'reference_price_ask_price_imb',
 'reference_price_bid_price_imb',
 'reference_price_wap_imb',
 'far_price_near_price_imb',
 'far_price_wap_imb',
 'near_price_wap_imb',
 'ask_price_bid_price_imb',
 'bid_price_wap_imb',
 'ask_price_bid_price_wap_imb2',
 'ask_price_bid_price_reference_price_imb2',
 'ask_price_wap_reference_price_imb2',
 'bid_price_wap_reference_price_imb2',
 'matched_size_bid_size_ask_size_imb2',
 'matched_size_bid_size_imbalance_size_imb2',
 'matched_size_ask_size_imbalance_size_imb2',
 'bid_size_ask_size_imbalance_size_imb2',
 'wap_momentum',
 'imbalance_momentum',
 'price_spread',
 'spread_intensity',
 'price_pressure',
 'market_urgency',
 'depth_pressure',
 'spread_depth_ratio',
 'mid_price_movement',
 'micro_price',
 'all_sizes_std',
 'all_sizes_skew',
 'all_sizes_kurt',
 'matched_size_shift_1',
 'matched_size_ret_1',
 'matched_size_shift_2',
 'matched_size_ret_2',
 'matched_size_shift_3',
 'matched_size_ret_3',
 'matched_size_shift_4',
 'matched_size_shift_5',
 'matched_size_shift_6',
 'matched_size_shift_7',
 'matched_size_shift_8',
 'matched_size_shift_9',
 'matched_size_shift_10',
 'imbalance_size_shift_1',
 'imbalance_size_ret_1',
 'imbalance_size_shift_2',
 'imbalance_size_ret_2',
 'imbalance_size_shift_3',
 'imbalance_size_ret_3',
 'imbalance_size_shift_4',
 'imbalance_size_ret_4',
 'imbalance_size_shift_5',
 'imbalance_size_ret_5',
 'imbalance_size_shift_6',
 'imbalance_size_ret_6',
 'imbalance_size_shift_7',
 'imbalance_size_ret_7',
 'imbalance_size_shift_8',
 'imbalance_size_ret_8',
 'imbalance_size_shift_9',
 'imbalance_size_ret_9',
 'imbalance_size_shift_10',
 'imbalance_size_ret_10',
 'reference_price_shift_1',
 'reference_price_ret_1',
 'reference_price_shift_2',
 'reference_price_ret_2',
 'reference_price_shift_3',
 'reference_price_ret_3',
 'reference_price_shift_4',
 'reference_price_ret_4',
 'reference_price_shift_5',
 'reference_price_ret_5',
 'reference_price_shift_6',
 'reference_price_ret_6',
 'reference_price_shift_7',
 'reference_price_ret_7',
 'reference_price_shift_8',
 'reference_price_ret_8',
 'reference_price_shift_9',
 'reference_price_ret_9',
 'reference_price_shift_10',
 'reference_price_ret_10',
 'imbalance_buy_sell_flag_shift_1',
 'imbalance_buy_sell_flag_ret_1',
 'imbalance_buy_sell_flag_shift_2',
 'imbalance_buy_sell_flag_ret_2',
 'imbalance_buy_sell_flag_shift_3',
 'imbalance_buy_sell_flag_ret_3',
 'imbalance_buy_sell_flag_shift_4',
 'imbalance_buy_sell_flag_ret_4',
 'imbalance_buy_sell_flag_shift_5',
 'imbalance_buy_sell_flag_ret_5',
 'imbalance_buy_sell_flag_shift_6',
 'imbalance_buy_sell_flag_ret_6',
 'imbalance_buy_sell_flag_shift_7',
 'imbalance_buy_sell_flag_ret_7',
 'imbalance_buy_sell_flag_shift_8',
 'imbalance_buy_sell_flag_ret_8',
 'imbalance_buy_sell_flag_shift_9',
 'imbalance_buy_sell_flag_ret_9',
 'imbalance_buy_sell_flag_shift_10',
 'imbalance_buy_sell_flag_ret_10',
 'ask_price_diff_1',
 'ask_price_diff_2',
 'ask_price_diff_3',
 'ask_price_diff_4',
 'ask_price_diff_5',
 'ask_price_diff_6',
 'ask_price_diff_7',
 'ask_price_diff_8',
 'ask_price_diff_9',
 'ask_price_diff_10',
 'bid_price_diff_1',
 'bid_price_diff_4',
 'bid_price_diff_5',
 'bid_price_diff_6',
 'bid_price_diff_7',
 'bid_price_diff_8',
 'bid_price_diff_9',
 'bid_price_diff_10',
 'ask_size_diff_1',
 'ask_size_diff_2',
 'ask_size_diff_3',
 'ask_size_diff_4',
 'ask_size_diff_5',
 'ask_size_diff_6',
 'ask_size_diff_7',
 'ask_size_diff_8',
 'ask_size_diff_9',
 'ask_size_diff_10',
 'bid_size_diff_1',
 'bid_size_diff_2',
 'bid_size_diff_3',
 'bid_size_diff_4',
 'bid_size_diff_5',
 'bid_size_diff_6',
 'bid_size_diff_7',
 'bid_size_diff_8',
 'bid_size_diff_9',
 'bid_size_diff_10',
 'weighted_wap_diff_1',
 'weighted_wap_diff_2',
 'weighted_wap_diff_3',
 'weighted_wap_diff_4',
 'weighted_wap_diff_5',
 'weighted_wap_diff_6',
 'weighted_wap_diff_7',
 'weighted_wap_diff_8',
 'weighted_wap_diff_9',
 'price_spread_diff_1',
 'price_spread_diff_2',
 'price_spread_diff_3',
 'price_spread_diff_4',
 'price_spread_diff_5',
 'price_spread_diff_6',
 'price_spread_diff_7',
 'price_spread_diff_8',
 'price_spread_diff_9',
 'price_spread_diff_10',
 'price_change_diff_2',
 'size_change_diff_2',
 'price_change_diff_3',
 'size_change_diff_3',
 'price_change_diff_4',
 'size_change_diff_4',
 'price_change_diff_5',
 'size_change_diff_5',
 'price_change_diff_6',
 'size_change_diff_6',
 'price_change_diff_7',
 'size_change_diff_7',
 'price_change_diff_8',
 'size_change_diff_8',
 'price_change_diff_9',
 'size_change_diff_9',
 'price_change_diff_10',
 'size_change_diff_10',
 'rolling_diff_ask_price_2',
 'rolling_std_diff_ask_price_2',
 'rolling_std_diff_bid_price_2',
 'rolling_diff_ask_size_2',
 'rolling_std_diff_ask_size_2',
 'rolling_diff_bid_size_2',
 'rolling_std_diff_bid_size_2',
 'rolling_diff_ask_price_3',
 'rolling_std_diff_ask_price_3',
 'rolling_diff_bid_price_3',
 'rolling_std_diff_bid_price_3',
 'rolling_diff_ask_size_3',
 'rolling_std_diff_ask_size_3',
 'rolling_diff_bid_size_3',
 'rolling_std_diff_bid_size_3',
 'rolling_diff_ask_price_4',
 'rolling_std_diff_ask_price_4',
 'rolling_diff_bid_price_4',
 'rolling_diff_ask_size_4',
 'rolling_std_diff_ask_size_4',
 'rolling_diff_bid_size_4',
 'rolling_std_diff_bid_size_4',
 'rolling_diff_ask_price_5',
 'rolling_std_diff_ask_price_5',
 'rolling_diff_bid_price_5',
 'rolling_std_diff_bid_price_5',
 'rolling_diff_ask_size_5',
 'rolling_std_diff_ask_size_5',
 'rolling_diff_bid_size_5',
 'rolling_std_diff_bid_size_5',
 'rolling_diff_ask_price_6',
 'rolling_std_diff_ask_price_6',
 'rolling_diff_bid_price_6',
 'rolling_std_diff_bid_price_6',
 'rolling_diff_ask_size_6',
 'rolling_std_diff_ask_size_6',
 'rolling_diff_bid_size_6',
 'rolling_std_diff_bid_size_6',
 'rolling_diff_ask_price_7',
 'rolling_std_diff_ask_price_7',
 'rolling_diff_bid_price_7',
 'rolling_std_diff_bid_price_7',
 'rolling_diff_ask_size_7',
 'rolling_std_diff_ask_size_7',
 'rolling_diff_bid_size_7',
 'rolling_std_diff_bid_size_7',
 'rolling_diff_ask_price_8',
 'rolling_std_diff_ask_price_8',
 'rolling_diff_bid_price_8',
 'rolling_std_diff_bid_price_8',
 'rolling_diff_ask_size_8',
 'rolling_std_diff_ask_size_8',
 'rolling_diff_bid_size_8',
 'rolling_std_diff_bid_size_8',
 'rolling_diff_ask_price_9',
 'rolling_std_diff_ask_price_9',
 'rolling_diff_bid_price_9',
 'rolling_std_diff_bid_price_9',
 'rolling_diff_ask_size_9',
 'rolling_std_diff_ask_size_9',
 'rolling_diff_bid_size_9',
 'rolling_std_diff_bid_size_9',
 'rolling_diff_ask_price_10',
 'rolling_std_diff_ask_price_10',
 'rolling_diff_bid_price_10',
 'rolling_diff_ask_size_10',
 'rolling_std_diff_ask_size_10',
 'rolling_diff_bid_size_10',
 'rolling_std_diff_bid_size_10',
 'mid_price*volume',
 'harmonic_imbalance',
 'seconds',
 'minute',
 'time_to_market_close',
 'global_median_size']

# **Submission**

In [13]:
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices) / np.sum(std_error)
    out = prices - std_error * step
    return out

if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64
    qps, predictions = [], []
    cache = pd.DataFrame()

    # Weights for each fold model
    if CAT:
        #feature_columns = list(df_train_feats.columns)
        
        #cbt_model_weights = weighted_average(models_cbt)
        models=[]
        for i in range(1,6):
            model_filename = f"/kaggle/input/cattt3/doblez_{i}.model"
            m = joblib.load(model_filename)
            models.append(m)
        model_filename = "/kaggle/input/cattt3/total_1.model"
        m = joblib.load(model_filename)
        models.append(m)
        cat_model_weights = weighted_average(models)
        #cat_model_weights=[0.3,0.3]
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        cache = pd.concat([cache, test], ignore_index=True, axis=0)
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        feat = generate_all_features(cache)[-len(test):]
        print(f"Feat Shape is: {feat.shape}")

        # Generate predictions for each model and calculate the weighted average
        if CAT:
            cat_predictions = np.zeros(len(test))
            for model, weight in zip(models, cat_model_weights):
                cat_predictions += weight * model.predict(feat[select_feature])

        predictions = cat_predictions
        
        #Using mean predictions rather than zero sum
        final_predictions = predictions - np.mean(predictions)
        clipped_predictions = np.clip(final_predictions, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))

    time_cost = 1.146 * np.mean(qps)
    print(f"The code will take approximately {np.round(time_cost, 4)} hours to reason about")


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
10 qps: 2.0165146827697753
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
20 qps: 1.9265484452247619
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
Feat Shape is: (200, 305)
30 qps: 1.9030784924825033
Feat Shape is: